In [ ]:
from DataIngestor import DataIngestor
from SharpeSurfaceGenerator import SharpeSurfaceGenerator
from ETFRanker import ETFRanker

from datetime import datetime
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# logReturns = 1 means log returns will be used in the calculation of portfolio returns, 0 means pct_changes
# Aperiods = lookback, determines what counts as the short term period, 20 days is the Default setting
# the selection of the ETF is based on maximum weighted score of:

# A returns, or returns in the short term period

# B returns, or returns in the longer term period (here set to 3 times the short term period)

# volatility, or volatility in the short term period.

# Frequency= holding period, how long to wait to rebalance, 18W-FRI is the Default setting for SPY-TLT combination.
# The frequency of a given ETF list can be determined by generating a Sharpe surface, assuming Fridays is the best trading day.

# Frequency="B" every business day, "D" every calendar day; "10B" means every 10 business days ("2B" is the minimum)
# Frequency="W" every weeek, "2W" for every 2 weeks, "3W" every 3 weeks etc
# Frequency="W-TUE" every Tuesday, "2W-TUE" for every 2 Tuesdays, "3W-TUE" every 3 Tuesdays etc
# Frequency= "BM" every month, "2BM" for every 2 months, "3BM" every 3 months etc; B relates to business days; 31 or previous business day if necessary
# Frequency="SM" on the middle (15) and end (31) of the month, or previous business day if necessary

# Delay = 1 if the trade occurs instantaneously with the signal, 2 if the trade occurs 1 day after the signal

# ShortTermWeight = the weight of short term returns, .3 by default
# LongTermWeight = the weight of long term returns, .4 by default
# ShortTermVolatilityWeight = the weight of short term volatility, .4
# ShortTermWeight+LongTermWeight+ShortTermVolatilityWeight should add up to 1.

# other parameters (experimental)
# Cash Filter = 0 (or 1) forces the portfolio to invest in whatever ETF in the list "StandsForCash"
# MAperiods = moving average, usually 200, that prices have to be above of to allow trading, if the cash filter is set to 1.
# StandsForCash = "TLT" #what the system invests in when the CashFilter is triggered, the rightmost ETF in the dataframe
# if you set cash filter to 1, you need to identify what ETF StandsForCash (should be the rightmost ETF in the dataframe)
# Zboundary = -1.5 #alternative cash filter (but need understand and to uncomment line: alternative cash filter)
# Zperiods = 200  #alternative cash filter  (but need understand and to uncomment line: alternative cash filter)
# momentum = 1 means A and B returns are ranked in increasing order (momentum), 0 in decreasing order (reversion to the mean)
# volmomentum = 1 volatility ranked in increasing order (momentum), 0 in decreasing order (reversion to the mean)


In [ ]:
##############
# PARAMETERS #
##############

# momentum --> experimental mean reversion trading
# CashFilter, MAperiods, StandsForCash --> for the experimental cash filter
# Aperiods = lookback, determines what counts as the short term period, 20 days is the Default setting
# frequency = holding

param_dict = {"logReturns": 0,
"Aperiods": 6 * 5, 
"Frequency": "14W-FRI",
"momentum" : 1,
"CashFilter" : 0,
"MAperiods" : 200,
"Delay" : 1,
"ShortTermWeight" : 0.3,
"LongTermWeight" : 0.4,
"ShortTermVolatilityWeight" : 0.4,
"StandsForCash" : "SHY",
"max_holding" : 20,
"step_holding" : 1,
"max_lookback" : 12,
"step_lookback" : 5,
"Zboundary": -1.5,
"Zperiods": 200,}



start_year = 2018
start_date= datetime(start_year, 1, 1)
#end_date= datetime(2022, 12, 31)
end_date= datetime.now()
canada_etfs = ["FDN.TO",
"FBT.TO",
"ZEO.TO",
"SKYY.TO",
"XHC.TO",
"FHH.TO",
"ZGEN.TO",
"EARK.NE",
"CIF.TO",
"FHG.TO",
"TRVL.TO",
"FHQ.TO",
"NXTG.TO",
"ZBK.TO",
"RUBY.TO",
"FLI.TO",
"XST.TO",
"COMM.TO",
"CHPS.TO",
"TGRE.TO",
"XGB.TO"]



canada_bonds = [
    "VSC.TO",
    "VSB.TO",
    "ZIC.TO",
    "HBB.TO",
    "TDB.TO",
    "XBB.TO",
    "XGB.TO"]

stock_list = canada_etfs

In [ ]:
# ingest data from yahoo finance
data_ingestor = DataIngestor(stock_list, start_date, end_date)

In [ ]:
# set dataframes from ingested data
close_df, adj_close_df = data_ingestor.close_df_adj_close_df()

In [ ]:
# Generate sharpe surface to find optimal lookback and holding period. 
# use the max_lookback and max_holding paramters to adjust the limits of the graph
sharpe_gen = SharpeSurfaceGenerator(close_df, adj_close_df, param_dict)

In [ ]:
sharpe_gen.generate_sharpe_matrix()

In [ ]:
#plot Sharpe surface
sharpe_gen.plot_sharpe_surface()

In [ ]:
# rank ETFs to make a choice for today/end_date
ranker = ETFRanker(close_df, adj_close_df, param_dict)

In [ ]:
# choose highest rank
ranker.rank_etfs()

In [ ]:
# The Rotational_Momentum_Simulator will show the returns over a given trading period.

# July 28, 2023 - ETF lookback: 6, holding 14